In [1]:
# To test slideshow run this cell.  Use LiveReveal.js in class.
!jupyter nbconvert --to slides --post serve Lecture-17-Explicit-Finite-Difference.ipynb

[NbConvertApp] Converting notebook Lecture-17-Explicit-Finite-Difference.ipynb to slides
[NbConvertApp] Writing 283205 bytes to Lecture-17-Explicit-Finite-Difference.slides.html
[NbConvertApp] Serving local reveal.js
Serving your slides at http://127.0.0.1:8000/Lecture-17-Explicit-Finite-Difference.slides.html
Use Control-C to stop this server
^C

Interrupted


# Lecture 17:  Numerical Solutions to the Diffusion Equation (Explicit Methods)

### Sections

* [Introduction](#Introduction)
* [Learning Goals](#Learning-Goals)
* [On Your Own](#On-Your-Own)
    * [Taylor Series and Derivatives](#Taylor-Series-and-Derivatives)
    * [Dividing Space and Time](#Dividing-Space-and-Time)
* [In Class](#In-Class)
    * [Set up an initial condition](#Set-up-an-initial-condition)
    * [Set up other simulation parameters](#Set-up-other-simulation-parameters)
    * [Set up the data structure](#Set-up-the-data-structure)
    * [Write the solver](#Write-the-solver)
    * [Plot the results](#Plot-the-results)
* [Homework](#Homework)
* [Summary](#Summary)
* [Looking Ahead](#Looking-Ahead)
* [Reading Assignments and Practice](#Reading-Assignments-and-Practice)

### Introduction
----

Tomes have  been written on numerical methods. We will explore four methods in the next four lectures. If you can map previous learning elements/topics into each method then you are doing well.

If the material in this lecture seems incomplete, it is. There is much more to learn and my intent is to help you see "under the hood". All the concepts here are carried over into more modern and more
complex numerical methods.

[Top of Page](#Sections)

### Learning Goals
----

You will learn four different ways to solve the diffusion equation in the next four lectures.  This is a chance to practice what you learned in previous lectures within each of the worked out problems.  Things to look out for:

* Taylor's expansions
* Linear algebra
* Fourier series
* Boundary conditions

[Top of Page](#Sections)

### On Your Own
----

* Review the taylor series
* Review the idea of grids

#### Taylor Series and Derivatives

As a reminder, the Taylor series is used to develop approximations of derivative quantities from finite differences.  The Taylor polynomial is the basis for the finite difference calculus.  These definitions were in Lecture 10.

Taylor's approximation of a first derivative:

$$
- 2 h \left. \frac{d}{d \xi_{1}} f{\left (\xi_{1} \right )} \right|_{\substack{ \xi_{1}=c }} - f{\left (c - h \right )} + f{\left (c + h \right )} = 0
$$

And the second derivative as a central difference:

$$
h^{2} \left. \frac{d^{2}}{d \xi_{1}^{2}}  f{\left (\xi_{1} \right )} \right|_{\substack{ \xi_{1}=c }} + 2 f{\left (c \right )} - f{\left (c - h \right )} - f{\left (c + h \right )} = 0
$$

[Top of Page](#Sections)

#### Dividing Space and Time

In order that we may use our Taylor's series approximations for the time and space derivatives we need to define the domain of our simulation.  One easy way to visualize time and space descretization is to use a "grid" construction.  The figure below shows one way that the time and space variables can be represented.  

In this definition the grid is a _uniform_ grid in each independent variable - meaning that the distance in the independent variable between grid points is the same.

![](http://ridcully.mat.rpi.edu/4960/index.php?n=Main.Lectures?action=download&upname=grid.png)

This image is from [this page](https://me.ucsb.edu/~moehlis/APC591/tutorials/tutorial5/node3.html) and [this tutorial](https://me.ucsb.edu/~moehlis/APC591/tutorials/tutorial5/tutorial5.html) and is included as a link on the course server.

[Top of Page](#Sections)

### In Class
----

In this part we will solve the diffusion equation numerically and write our own numerical finite-difference solver.  The steps we need to take are:

* Set up an initial condition
* Write a finite-difference representation of our PDE
* Iterate
* Store the results
* Visualize the results

#### Set up an initial condition

We need to set up a few parameters before we get started:

* `numberOfPoints` - the number of grid points within our computational domain
* `lengthOfDomain` - the physical length of our computational domain
* `dx` - the distance between successive grid points in our domain
* `xPoints` - a linear space divided into numberOfPoints points.
* `initialCondition` - our starting distribution of solute (i.e. $c(x,0)$).

In [1]:
# Run this cell before lecture

%matplotlib osx
import numpy as np
import matplotlib.pyplot as plt

We start with an initial seed of data. This could be anything you like - any initial condition.  Unlike our other analytical strategies there are no coefficients to compute, no functions to fit - you can just put your data into an array. 

In [2]:
numberOfPoints = 100
lengthOfDomain = 1.0
dx = lengthOfDomain/numberOfPoints
xPoints = np.linspace(0.0, lengthOfDomain, numberOfPoints)
initialCondition = np.sin(xPoints*np.pi/lengthOfDomain)

NameError: name 'np' is not defined

Here we choose a `sin` function as our initial condition.  Really - it can be anything based on the physical problem you are solving.

In [3]:
def plotIC():
    fig = plt.figure()
    axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
    axes.plot(xPoints, initialCondition, 'ro')
    axes.set_xlabel('Distance $x$')
    axes.set_ylabel('Concentration of Stuff $c(x,t)$')
    axes.set_title('Initial Conditions');

In [ ]:
plotIC()

One thing I have glossed over from the start of this class is the need to scale your equations
appropriately. You may be asking yourself at this point, why N points above? One of the concepts in numerical solutions is that you descritize space and time. So all we know so far is that the initial condition above is broken into `numberOfPoints` equivalent units of space. (It would be natural to ask at this point, must they be equivalent? The reality is that you can have unstructured grids and this is very important for real engineering problems. At this point we leave the grid structured so that other parts of the problem are easier to deal with.)

#### Set up other simulation parameters

We now set:

* `diffusionCoefficient` - the diffusion coefficient
* `dt` - the discrete time step (you'll note the formulaic choice of `dt`)
* `numberOfIterations` - the number of iterations we will take in the solution of the PDE (e.g. the total time of the simulation will be `numberOfIterations` multiplied by `dt`)

In [ ]:
diffusionCoefficient = 10.0
dt = dx**2/(4*diffusionCoefficient)
numberOfIterations = 1000

Our choice of timestep is restricted thus:

$$
dt \leq \frac{\Delta x^2}{2 \, D}
$$

There is a good discussion of an error analysis in "Numerical Recipes" and other numerical methods text will go through the details of a stability analysis.  Other texts (e.g. Hornbeck) just state the above as true.  When we solve the PDE - try and make $dt$ smaller, equal to and larger than the Courant stability limit.  See what happens!

#### Set up the data structure

A short discussion on our data structure is needed here.  There are three strategies that I have used in solving PDEs.  One is to store ALL the data, another is to store SOME of the data, and the last is to store NONE of the data but the very last computation.  Each strategy has advantages and disadvantages.  At this point in your education - all strategies probably seem equally hard.  For now - let us design a data structure that stores all the data with the caveat that things are rarely done using the method I'll show you.

Let us create a `numpy` array that has one dimension equal to the number of points in our grid and another dimension that is equal to the number of iterations we wish to take.  

In [ ]:
arrayWithAllTheData = np.zeros((numberOfPoints,numberOfIterations), dtype='float32')

#### Set up the initial condition

You can think of the 2D array as having one space axis (the first index, we will use `i` for this one) and one time axis (the second index, we will use `j` for this one).

We will set our initial conditions by assigning the `initialCondition` array to the first row of the `arrayWithAllTheData`.  Note the slicing in the first index.

In [ ]:
arrayWithAllTheData[:,0] = initialCondition

With our initial conditions in place we need to develop the computational steps to advance our solution in time.  The PDE we are solving (with a constant diffusion coefficient) is:

$$
\frac{\partial c(x,t)}{\partial t} = D \frac{\partial^2 c(x,t)}{\partial x}
$$

In the next step we assign the spatial coordinate an index $i$ and a time coordinate an index $j$.  The finite difference representation of our PDE with $u_{i,j}$ as the stored value that represents $c(x,t)$ is:

$$
\frac{u_{i,\, j+1} - u_{i,\, j}}{\Delta t} = D \frac{u_{i - 1,\, j} - 2 u_{i,\, j} + u_{i + 1,\, j}}{\Delta x^2}
$$

We can algebraically solve for $u_{i+1,\, j}$:

$$
u_{i,\, j+1} = \frac{D \Delta t}{\Delta x^2} \left( u_{i - 1,\, j} - 2 u_{i,\, j} + u_{i + 1,\, j} \right) + u_{i,\, j}
$$

From the expression above you can see that all the terms on the RHS of the expression are at the index $j$ (the last iteration) and all the terms on the LHS are for the $j+1$ index (the next iteration).  This scheme defines a simple method (with a restrictive timestep) for integrating a PDE.  We will apply this method below.

To make all of this work we have to proceed as follows:

* Compute the prefactor $D \Delta t/ \Delta x^2$.
* Apply the boundary conditions in the $j$th row of the array.
* Using the $j$ row of the array (plus the boundary conditions), fill in row $j+1$ of the array with values corresponding to the new time $t + \Delta t$ according to the equation above.
* Advance the index and repeat until all rows are filled.
* Visualize the results.

**Advancing the index** and **repeating until the rows are filled** are done in the cell below:

#### Write the solver

In [ ]:
for j in range(1,numberOfIterations):
    for i in range(1,numberOfPoints-1):
        arrayWithAllTheData[i,j] = # What should you put here?

#### DIY:  Sketch the algorithm up to this point and for the cell below.

Doing this will help you visualize the operations and it will increase your ability to make modifications in the future and devise new more compact ways to integrate this PDE.

If you've sketched the algorithm as advised above then you see that in our development of this solution we implicitly set the boundary conditions.  We initialize `arrayWithAllTheData` with `np.zeros` and then compute on all the interior rows/columns.  This creates a condition where all the boundary cells are set to zero and their values remain untouched throughout the computation.

#### Plot the results

In [ ]:
%matplotlib inline
import numpy as np
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt

def plotArray(xPoints, dataArray, rowID=0):
    """
    This function in conjunction with interact() permits
    inspection of the contents of an array, row by row.  This
    is useful for some small tasks such as examining the results
    of a PDE solution.
    """
    x = xPoints
    y = dataArray[:,rowID]
    fig = plt.figure(figsize=(10,8))
    axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
    axes.set_ylim(0,1)
    axes.plot(x, y, 'ro', label=r"$c(x,t)$")
    axes.legend()
    axes.grid(False)
    return

In [ ]:
interact(plotArray, 
         xPoints=fixed(xPoints), 
         dataArray=fixed(arrayWithAllTheData),
         rowID=(0,numberOfIterations-1,1), );

#### DIY:  Compute a solution where you change the boundary conditions on the LHS to be $c(x=L,t) = 1.0$.

In [ ]:
# Your solver code goes here.

#### Optional DIY:  Vectorize the above solution method.


In [ ]:
# Your solver code goes here.

[Top of Page](#Sections)

### Homework 

(Optional) 

----

#### Part 1:  Solve the following problem:

$$
\frac{\partial c(x,t)}{\partial t} = D \frac{\partial^2 c(x,t)}{\partial x^2}
$$

with the initial condition 

$$c(x,t=0) = 1.0,$$

over the domain 

$$( x \, \vert \, 0 \le x \le 1.0 ).$$

with zero flux boundary conditions.

#### Part 2:  Solve the following problem:

$$
\frac{\partial c(x,t)}{\partial t} = \frac{\partial}{\partial x} D(c) \frac{\partial c(x,t)}{\partial x}
$$

with the initial condition:

$$
c(x \leq 0.5 , t=0) = 1.0\\
c(x \gt 0.5 , t=0) = 0.0
$$

with D having the dependence:

$$
D(c) = D_0\cdot c \cdot(1-c)
$$

over the domain:

$$
( x \, \vert \, 0 \le x \le 1.0 )
$$

with zero flux boundary conditions.

### Homework Notes
----

If you've got the above all worked out.  The first homework problem amounts to changing the initial conditions.  For the second problem choose the Courant condition (the condition that sets the maximium time step for which the finite difference method is stable) such that the time step is chosen for the most restrictive case.

[Top of Page](#Sections)

### Looking Ahead
----

[Top of Page](#Sections)

### Reading Assignments and Practice
----

[Top of Page](#Sections)